In [1]:
import pandas as pd
from dotenv import load_dotenv
from googlemaps import convert
import googlemaps
import os
import requests
import json
import foursquare
from config import dic_categorias
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import matplotlib 

In [2]:
load_dotenv()

True

In [3]:
tokg = os.getenv("tokg")
tokf3 = os.getenv ("tokf3")
client2= MongoClient ("localhost:27017")

In [5]:
client = foursquare.Foursquare( access_token= tokf3 )
gmaps = googlemaps.Client(key= tokg )
db = client2.get_database("ciudades")

# he borrado todo lo que tenia de datasets, y empiezo directamente con lo que sería el flujo de la api

In [34]:
dic_categorias= {
    
    "restaurantes": ["restaurant"],
    "playas": "4bf58dd8d48988d1e2941735",
    "ocio_nocturno": ["pub","Cocktail Bar"],
    "hoteles": ["hotel"],
    "ocio_deporte": ["gym", "Fitness"],
    "colegios" : ["school", "primary school", "elemental school"],
    "guarderias" : ["kindergarden", "preschool"],
    "universidades": ["college", "university"],
    "aeropuertos" : ["airport"],
    "estaciones de tren": ["train station"],
    "parques": ["park"],
    "hospitales" : ["hospital", "medical center"],
    "cines": ["cinema", "movie theatre"],
    "teatros": ["theatre"],
    "museos": ["museum"],
    "salas_de_conciertos": ["rock club", "jazz club", "concert"],
    "piscina_para_Emilio": ["pool"],
    "bibliotecas" : ["library", "book store"],               
    "centros_comerciales": ["mall", "shopping center"]
    
                                            }

input: ciudad y pais

In [177]:
test_and_create ("buenos aires","argentina")

'se han introducido los campos'

In [171]:
test_and_create ("madrid", "españa")

'Ya está creada'

In [407]:
a[0]["formatted_address"]

'Calle de Avefría, 28025 Madrid, Spain'

In [6]:
def test (nombre):
    lista= db.list_collection_names()
    return nombre in lista       

In [7]:
def test_and_create( ciudad, pais):
    try: 
        busqueda= ciudad + ", " + pais #unimos en un string para pasarselo como búsqueda a gplaces
        coords=gmaps.geocode( address= busqueda)
        nombre = coords[0]['formatted_address'] # nos quedamos con el nombre, que será el de la colección
        a=coords[0]["geometry"]["location"]["lat"] 
        b=coords[0]["geometry"]["location"]["lng"]
        c = str (a) +"," + str(b) # Formato que luego entienda foursquare
        dic= {"nombre": nombre,
                "coordenadas": c,
                 "latitud":a,
                 "longitud":b}

            
        if test(nombre) == True:
            
            return (nombre)
        
        else:
            
            pass
        
            db.create_collection (nombre)   
    except: "los datos introducidos no son correctos"
        
    return  get_coords (nombre, dic)

In [8]:
def get_coords(nombre, dic):
    collection= nombre 
    db[collection].insert_one (dic)
    coord= dic["coordenadas"]
    return get_data_fields (coord, collection)
        

In [9]:
def get_data_fields (coord, collection):
    tokf3 = os.getenv ("tokf3")
    client = foursquare.Foursquare( access_token= tokf3 )
    cantidad=[]
    dic_insert={}
    for categoria, codigo in dic_categorias.items():  
        request = client.venues.search(params = {'query': codigo , 'll': coord, "radius": 1500, "limit": 50})
        lista=request["venues"]
        cantidad= len (lista)
        dice1={categoria: {"número":cantidad }}
        dice=[]
        for i in range (0,len (lista)): # para cada venue, sacame:
            nombre = lista[i]["name"]
            latitud= lista[i]["location"]["lat"]
            longitud= lista[i]["location"]["lng"]
            try:
                tipo = lista[i]["categories"][0]["name"]
            except:
                pass
            try:
                direccion= lista[i]["location"]["formattedAddress"]
            except:
                pass
        
            dice2 = {"nombre": nombre, 
                       "latitud":latitud,
                       "longitud": longitud,
                        "tipo" : tipo,
                        "direccion": direccion }
            dice.append (dice2)          
        
            dice1[categoria].update ({"listado": dice})
        db[collection].insert_one (dice1)
    
    return "se han introducido los campos"

In [10]:
def get_data_fields (coord, collection):
    tokf3 = os.getenv ("tokf3")
    client = foursquare.Foursquare( access_token= tokf3 )
    dic_categorias= {
    
    "restaurantes": ["restaurant"],
    "playas": "4bf58dd8d48988d1e2941735",
    "ocio_nocturno": ["pub","Cocktail Bar"],
    "hoteles": ["hotel"],
    "ocio_deporte": ["gym", "Fitness"],
    "colegios" : ["school", "primary school", "elemental school"],
    "guarderias" : ["kindergarden", "preschool"],
    "universidades": ["college", "university"],
    "aeropuertos" : ["airport"],
    "estaciones de tren": ["train station"],
    "parques": ["park"],
    "hospitales" : ["hospital", "medical center"],
    "cines": ["cinema", "movie theatre"],
    "teatros": ["theatre"],
    "museos": ["museum"],
    "salas_de_conciertos": ["rock club", "jazz club", "concert"],
    "piscina_para_Emilio": ["pool"],
    "bibliotecas" : ["library", "book store"],               
    "centros_comerciales": ["mall", "shopping center"]                                                     
            }
    for categoria, codigo in dic_categorias.items():  
        request = client.venues.search(params = {'query': codigo , 'll': coord, "radius": 1500, "limit": 50})
        lista=request["venues"]
        
        
        dice=[]
        for i in range (0,len (lista)): # para cada venue, sacame:
            nombre = lista[i]["name"]
            latitud= lista[i]["location"]["lat"]
            longitud= lista[i]["location"]["lng"]
            try:
                tipo = lista[i]["categories"][0]["name"]
            except:
                tipo= "no definido"
            try:
                direccion= str (lista[i]["location"]["formattedAddress"])[2:-2]
            except:
                direccion= "no disponible"
        
            dice2 = {"nombre": nombre,
                "categoria": categoria, 
               "latitud":latitud,
               "longitud": longitud,
                "tipo" : tipo,
               "direccion":direccion }
            dice.append (dice2)          
        
        try:   # cuando dice está vacio, dá error, así lo obviamos.
            db[collection].insert_many (dice)
        except:
            pass
    
    return (collection)

In [11]:
test_and_create ("clara campoamor", "madrid")

'Calle Clara Campoamor, 28025 Madrid, Spain'

Crear una función que haga una query de X restaurantes ok
Crear un dataframe con esa query
Ese dataframe, pasandolo a geodataframe
Con ese dataftame, plotearlo en folium

Lo creo directamente para que sea un segundo imput del usuario, de hecho puede ser el principal, y si le da error, le saco el otro para que cree la ciudad. collection e item serán desplegables, y el número será la cantidad que quiera que salga. 

In [390]:
def query(collection, items, numero):
    # ver si se pueden poner varios items, modificar funcion caso
    a=list (db[collection]. find ({item: {"$exists": True}},{ item +".listado" :1, "_id":0}))
    try:
        listapeque= a[0]["restaurantes"]["listado"][:numero]
    except:
        return "No hay resultados que mostrar"
        
    df=pd.DataFrame (listapeque)
    
    return making_map (df,collection)

In [505]:
collection= test_and_create ("avenida oporto", "madrid")

In [506]:
db[collection].find_one ()

{'_id': ObjectId('60a44d02bc723d4dbad85c8f'),
 'nombre': 'Av. Oporto, Madrid, Spain',
 'coordenadas': '40.3856358,-3.7250457',
 'latitud': 40.3856358,
 'longitud': -3.7250457}

In [27]:
def get_map (collection, items):
    lista=[]
    for item in items:
        a=list (db[collection]. find ({"categoria": item},{"latitud":1,"nombre":1, "longitud":1, "categoria":1, "_id":0}).limit(15))
        lista.append (a)
    cosa=[]
    for n in range (len(lista)):
        cosa+= lista[n]
    if len (cosa)==0:
        return False
    else:
        df=pd.DataFrame (cosa)
    
    return making_map (df, collection)

In [22]:
def making_map (df,collection):
    
    a=list (db[collection]. find ({"coordenadas": {"$exists": True}},{"latitud":1,"longitud" :1, "nombre":1,"_id":0})) 
    
    lat=a[0]["latitud"]
    
    long=a[0]["longitud"]
    
    nombre= a[0]["nombre"]
    
    mapa = folium.Map(location= [lat, long], zoom_start= 14, tooltip=nombre)
    for i, row in df.iterrows():
        
        marcador = {
            "location":[row["latitud"], row["longitud"]],
            "tooltip" : row["categoria"]
        }
    
        if row["categoria"] == "Festivo":
            icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
        
        elif row["categoria"] == "Vispera":
            icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "home",
                    icon_color = "black"
        )
        
        else:
            icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "briefcase",
                    icon_color = "black"
            )
        Marker(**marcador,icon = icon ).add_to(mapa)
    return mapa
    

In [587]:
def making_points (df, collection, mapa):
    for i, row in df.iterrows():
        
        marcador = {
            "location":[row["latitud"], row["longitud"]],
            "tooltip" : row["categoria"]
        }
    
        if row["categoria"] == "Festivo":
            icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
        
        elif row["categoria"] == "Vispera":
            icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "home",
                    icon_color = "black"
        )
        
        else:
            icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "briefcase",
                    icon_color = "black"
            )
        Marker(**marcador,icon = icon ).add_to(mapa)
    return mapa

In [38]:
collection=test_and_create ("avenida de oporto", "madrid")

In [25]:
items= ["restaurantes", "colegios", "piscinas"]

In [30]:
mapa=get_map (collection, items)

In [ ]:
Estructura streamlit:
    
    
    separo columnas:
        
        foto1                                   foto2
        
        
    "biennvenidos blablabla"
    
    "descripción en 10 palabras"
    
    separo columnas:
        
        dataframe con count de cosas                       barchar con  lo mismo                 
        # hacer función que coja 5 colleciones               # lo mismo
        # random para mostrar
        
        
    selecbox de collectios, items
                #1º modificar funciones para que compruebe aquí si existe
    
    mapa con cosas marcadas.
    
    
    dataframe con count
    
    
    

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

In [61]:
def grafica_post(collection):
    dic_categorias= {
    
    "restaurantes": ["restaurant"],
    "playas": "4bf58dd8d48988d1e2941735",
    "ocio_nocturno": ["pub","Cocktail Bar"],
    "hoteles": ["hotel"],
    "ocio_deporte": ["gym", "Fitness"],
    "colegios" : ["school", "primary school", "elemental school"],
    "guarderias" : ["kindergarden", "preschool"],
    "universidades": ["college", "university"],
    "aeropuertos" : ["airport"],
    "estaciones de tren": ["train station"],
    "parques": ["park"],
    "hospitales" : ["hospital", "medical center"],
    "cines": ["cinema", "movie theatre"],
    "teatros": ["theatre"],
    "museos": ["museum"],
    "salas_de_conciertos": ["rock club", "jazz club", "concert"],
    "piscina_para_Emilio": ["pool"],
    "bibliotecas" : ["library", "book store"],               
    "centros_comerciales": ["mall", "shopping center"]
    
                                            }
    c= list (dic_categorias.keys())
    lista=[]
    for item in c:
        a=list (db[collection].find ({"categoria": item},{ item :1, "_id":0}))
        cantidad= len (a)
        lista.append ( { collection : cantidad } )
    df=pd.DataFrame (lista, index=c)
    return df

In [49]:
import random

In [60]:
collection=random.choice (db.list_collection_names())
collection

'610 Purdue Mall, West Lafayette, IN 47907, USA'

In [64]:
grafica_intro()

,"2109 Broadway, New York, NY 10023, USA"
restaurantes,50
playas,0
ocio_nocturno,0
hoteles,0
ocio_deporte,0
colegios,0
guarderias,0
universidades,0
aeropuertos,0
estaciones de tren,0


In [58]:
prueba= pd.DataFrame (lista, index=c)

NameError: name 'lista' is not defined

In [293]:
prueba

,Madrid
restaurantes,50
playas,0
ocio_nocturno,13
hoteles,50
ocio_deporte,50
colegios,33
guarderias,0
universidades,8
aeropuertos,4
estaciones de tren,23


In [233]:
colecciones=db.list_collection_names ()

In [235]:
import random

In [238]:
ciudad= random.choice (db.list_collection_names ())

In [351]:
grafica_intro()

,Madrid
restaurantes,50
playas,0
ocio_nocturno,13
hoteles,50
ocio_deporte,50
colegios,33
guarderias,0
universidades,8
aeropuertos,4
estaciones de tren,23


In [336]:
c

['restaurantes',
 'playas',
 'ocio_nocturno',
 'hoteles',
 'ocio_deporte',
 'colegios',
 'guarderias',
 'universidades',
 'aeropuertos',
 'estaciones de tren',
 'parques',
 'hospitales',
 'cines',
 'teatros',
 'museos',
 'salas_de_conciertos',
 'piscina_para_Emilio',
 'bibliotecas',
 'centros_comerciales']

In [348]:
c= list (dic_categorias.keys())
lista=[]
for item in c:
    a=list (db[collection].find ({item: {"$exists": True}},{ item :1, "_id":0}))
    cantidad=a[0][item]["número"]
    lista.append ( {collection : cantidad } )
df=pd.DataFrame (lista, index=c)

In [349]:
df

,Madrid
restaurantes,50
playas,0
ocio_nocturno,13
hoteles,50
ocio_deporte,50
colegios,33
guarderias,0
universidades,8
aeropuertos,4
estaciones de tren,23


In [344]:
df=pd.DataFrame (lista, index=c)

In [345]:
df

,Madrid
restaurantes,50
playas,0
ocio_nocturno,13
hoteles,50
ocio_deporte,50
colegios,33
guarderias,0
universidades,8
aeropuertos,4
estaciones de tren,23


In [354]:
db.list_collection_names()

['Bande', 'CABA', 'Muy Muy', 'Dallas', 'Allariz', 'Madrid']

In [353]:
db["Cáceres\u200e"].drop()

In [356]:
st.selectbox ()

TypeError: selectbox() missing 2 required positional arguments: 'label' and 'options'

In [387]:
items = ["restaurantes", "playas", "ocio_nocturno", "hoteles"]

In [417]:
coord= "40.3835268, -3.7318703"

In [418]:
request = client.venues.search(params = {'query': "restaurante" , 'll': coord, "radius": 1500, "limit": 50})

In [419]:
request["venues"]

[{'id': '4b8ec7faf964a520c33633e3',
  'name': 'Restaurante Asiático Real',
  'location': {'lat': 40.373784061130195,
   'lng': -3.7328624725341797,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.373784061130195,
     'lng': -3.7328624725341797}],
   'distance': 1087,
   'cc': 'ES',
   'country': 'España'},
  'categories': [{'id': '4bf58dd8d48988d142941735',
    'name': 'Asian Restaurant',
    'pluralName': 'Asian Restaurants',
    'shortName': 'Asian',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1621368106',
  'hasPerk': False},
 {'id': '4d38a2300a288cfae335823c',
  'name': 'Bar Restaurante Los Extremeños Casa Ignacio',
  'location': {'address': 'C/ Hermanas Alonso, 2',
   'lat': 40.38746727714296,
   'lng': -3.738649910056414,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.38746727714296,
     'lng': -3.738649910056414}],
   'distance': 723,
   'postalCode': '28025'